In [324]:

from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
from scipy import signal
from glob import glob
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# constants:
SUBSAMPLE = 100  # scales the data by this factor
COLUMNS = ['HandStart', 'FirstDigitTouch',
        'BothStartLoadPhase', 'LiftOff',
        'Replace', 'BothReleased']
SUBJECTS = range(1, 2)
N_LABELS = 6

# Below are the paths to the data. Please pay attention to the % and * signs, these are needed in the loop.
TRAIN_DATA_PATH = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/Datasets/EEG/train/subj%d_series*_data.csv'
#The path below is for the test data used for a kaggle submission. This is not very relevant to our project.
# TEST_DATA_PATH = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/Datasets/EEG/test/subj%d_series*_data.csv'
# TRAIN_DATA_PATH = 'C:/Users/bas/Documents/MachineLearning/train/subj%d_series*_data.csv' #path on my laptop
# TEST_DATA_PATH =  'C:/Users/bas/Documents/MachineLearning/test/subj%d_series*_data.csv' 

SUBMISSION_FOLDER = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/'
SUBMISSION_NAME = 'subbmision_vu_48_sub_pca_4.csv'

PCA_COMPONENTS = 0.8
CUTT_OFF_FREQUENCY = 2
ORDER = 4
SAMPLE_FREQUENCY = 500

EPOCHS = 5
WINDOW_SIZE = 500

In [3]:
def prepare_training_data(data_path):
    data = pd.read_csv(data_path)
    event_path = data_path.replace('_data', '_events')
    labels = pd.read_csv(event_path)
    clean_data = data.drop(['id'], axis = 1)
    labels = labels.drop(['id'], axis = 1)
    return clean_data, labels

In [4]:
def prepare_test_data(data_path):
    data = pd.read_csv(data_path)
    return data

In [5]:
def scaler_transform(data, scaler):
    if scaler == None:
        scaler = StandardScaler()
        return scaler.fit_transform(data), scaler
    else:
        return scaler.transform(data)        

In [6]:
def butter_worth(data, cut_off, hz, order):
    filtered_data = np.empty(np.shape(data))
    wn = cut_off / (hz/2)
    b, a = signal.butter(order, wn, analog = False)
    for i in range(len(data[0])):
        filtered_data[:,i] = signal.lfilter(b, a, data[:,i])    
    return filtered_data

In [7]:
def pca_transform(data, n_components, pca_model):
    if pca_model == None:
        pca = PCA(n_components, whiten = True) 
        return pca.fit_transform(data), pca
    else:
        return pca_model.transform(data)

In [8]:
def read_training_data(train_data_paths):
    labels_raw = []
    features_raw = []
    for data_path in train_data_paths:
        data, labels = prepare_training_data(data_path)
        features_raw.append(data)
        labels_raw.append(labels)
    features_raw = pd.concat(features_raw)
    labels_raw = pd.concat(labels_raw)
    x_train = np.asarray(features_raw.astype(float))
    y_train = np.asarray(labels_raw.astype(float))
    return x_train, y_train

In [9]:
def read_test_data(test_data_paths):
    test_features_raw = []
    ids = []
    for data_path in test_data_paths:
        data = prepare_test_data(data_path)
        test_features_raw.append(data)
        ids.append(np.array(data['id']))
    test_features_raw = pd.concat(test_features_raw)
    ids = np.concatenate(ids)
    test_features_raw = test_features_raw.drop(['id'], axis = 1)
    x_test = np.asarray(test_features_raw.astype(float))
    return x_test, ids

In [10]:
def train_model(x_train, y_train, model, subsample):
    model.fit(x_train[::subsample,:], y_train[::subsample])
    return model

In [11]:
def make_prediction(x_test, model):
    prediction = model.predict_proba(x_test)[:,1]
    return prediction

In [12]:
def metric_auc_score(predictions, y_test, with_plot):
    scores = []
    legend_text = []
    for i in range(N_LABELS):
        fpr, tpr, _  = roc_curve(y_test[:,i], predictions[:,i], 1)
        scores.append(roc_auc_score(y_test[:,i], predictions[:,i]))
        legend_text.append(COLUMNS[i]+' (area = %.3f)' % (scores[i]))
        if with_plot == True:
            plt.plot(fpr, tpr)
    if with_plot == True:
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.legend(legend_text)
        plt.show()
    return scores

In [13]:
def single_metric_auc_score(predictions, y_test, with_plot):
    fpr, tpr, _  = roc_curve(y_test, predictions, 1)
    score = roc_auc_score(y_test, predictions)
    print(COLUMNS[0]+' AUC score = %.3f' % (score))
    if with_plot == True:
        plt.plot(fpr, tpr)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.show()
    return score

In [14]:
def all_auc_scores(prediction_total, test_data_total, subjects, with_plot):
    scores = []
    for subject in subjects:
        score = metric_auc_score(prediction_total[subject-1],
                                 test_data_total[subject-1], with_plot)
        scores.append(score)
        print('Mean AUC Score of Subject %d: %.3f' % \
              (subject, np.mean(score)))
    return scores

In [15]:
def make_submission_file(name, ids_total, prediction_total, columns, path):
    submission = pd.DataFrame(index = np.concatenate(ids_total), columns = columns,
                              data = np.concatenate(prediction_total))
    submission.to_csv(path+name, index_label = 'id', float_format = '%.3f')

In [274]:
def image_mappping(x_train, WINDOW_SIZE):
    result = []
    empty_matrix = np.atleast_3d(np.zeros(np.shape(x_train[0:WINDOW_SIZE])))
    for i in range(len(x_train)):
        if i-WINDOW_SIZE < 0:
            result.append(empty_matrix)
        else:
            result.append(np.atleast_3d(x_train[i-WINDOW_SIZE:i]))
    return result

In [257]:
def try_generator(x_train, y_train, WINDOW_SIZE):
    x = image_mappping(x_train, WINDOW_SIZE)
    while True:
        for image, task in zip(x, y_train):
            yield np.array([image]), np.array([task])


# def generator(x_train, y_train, image_size):
#     x_train = np.array_split(x_train, 10)
#     y_train = np.array_split(y_train, 10)
#     empty_matrix = np.expand_dims(np.zeros(np.shape(x_train[0][0:image_size])), axis=3)
#     while True:
#         for eeg, tasks in zip(x_train, y_train):
#             for i in range(len(eeg)):
#                 if i-image_size < 0:
#                     yield np.array([empty_matrix]), np.array([tasks[i]])
#                 else:
#                     yield np.array([np.expand_dims(eeg[i-image_size:i],axis=3)]), np.array([tasks[i]])

In [ ]:
def test_generator(x_test, WINDOW_SIZE):
    x = image_mappping(x_test, WINDOW_SIZE)
    while True:
        for image in x:
            yield np.array([image])

In [325]:
def class_weights(y_train):
    class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train[:,0]), y_train[:,0])
    return {0 : 1.,
    1: class_weight[1],
    2: class_weight[0]}

In [291]:
def init_cnn(window):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(window,32,1)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense((2)))
    model.add(Activation('softmax'))
    model.summary()
    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# def init_cnn(window):
#     model = Sequential()
#     model.add(Conv2D(16, kernel_size=(30, 4), activation='relu', input_shape=(window,32,1)))
#     model.add(Conv2D(32, (30, 4), activation='relu'))
#     model.add(MaxPool2D(pool_size=(200, 4)))
#     model.add(Dropout(0.25)) # Dropout 25% of the nodes of the previous layer during training
#     model.add(Flatten())     # Flatten, and add a fully connected layer
#     model.add(Dense(128, activation='relu')) 
#     model.add(Dropout(0.5))
#     model.add(Dense((2), activation='softmax')) # Last layer: 10 class nodes, with dropout
#     model.summary()
#     optimizer = Adam(lr=0.001)
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#     return model

In [167]:
prediction_total = []
test_data_total = []
ids_total = []
for subject in SUBJECTS:
    test_features_raw = []
    train_data_paths = glob(TRAIN_DATA_PATH % (subject))
#     test_data_paths =  glob(TEST_DATA_PATH % (subject))
  
    x_train, y_train = read_training_data(train_data_paths)  
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.5)
    
    x_train, scaler = scaler_transform(x_train, None)
    x_test = scaler_transform(x_test, scaler)    

#     predictions = np.empty((x_test.shape[0],6))
# #     Below you define the model you want to use.
#     logreg = LogisticRegression()
#     for i in range(N_LABELS):
#         print('Train subject %d, class %s' % (subject, COLUMNS[i]))
#         model = train_model(x_train, y_train[:,i], logreg, SUBSAMPLE)
#         predictions[:,i] = make_prediction(x_test, model)
#     test_data_total.append(y_test)   
#     prediction_total.append(predictions)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased


In [292]:
model = init_cnn(window=WINDOW_SIZE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 500, 32, 32)       320       
_________________________________________________________________
activation_14 (Activation)   (None, 500, 32, 32)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 498, 30, 32)       9248      
_________________________________________________________________
activation_15 (Activation)   (None, 498, 30, 32)       0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 124, 7, 32)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 124, 7, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 124, 7, 64)        18496     
__________

In [328]:
labels = to_categorical(y_train[:,0],num_classes = None)
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train[:,0]), y_train[:,0])
class_weights = {0 : 1.,
    1: class_weights[1],
    2: class_weights[0]}


{0: 1.0, 1: 18.17521083567595, 2: 0.5141441028525326}

In [ ]:
my_generater = try_generator(x_train, labels, WINDOW_SIZE)

model.fit_generator(my_generater, steps_per_epoch=len(x_train), epochs=20, class_weight=class_weights, verbose=1)

Epoch 1/20
 19513/711196 [..............................] - ETA: 2:06:00 - loss: 8.2719 - acc: 0.9716

In [296]:
my_test_generater = test_generator(x_test[::SUBSAMPLE], WINDOW_SIZE)
predictions = model.predict_generator(my_test_generater, steps=len(x_test))

In [308]:
labels_test = to_categorical(y_test[:,0],num_classes = None)

score = single_metric_auc_score(predictions[:, 1], labels_test[:,0], True)
score

ValueError: Found input variables with inconsistent numbers of samples: [711196, 7112]

In [ ]:
labels = to_categorical(y_train[:,0],num_classes = None)
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train[:,0]), y_train[:,0])
class_weights = {0 : 1.,
    1: class_weights[0],
    2: class_weights[1]}

my_generater = try_generator(x_train, labels, WINDOW_SIZE)

model.fit_generator(my_generater, steps_per_epoch=len(x_train), epochs=20, class_weight=class_weights, verbose=1)

In [ ]:
labels_test = to_categorical(y_test[:,0],num_classes = None)

score = single_metric_auc_score(predictions[:, 1], labels_test[:,0], True)
score

In [ ]:
# scores = all_auc_scores(prediction_total, test_data_total, SUBJECTS, with_plot=True)

In [ ]:
# print('Mean Columnwise AUC ROC Score: %.3f, %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
#########################KAGGLE SUBMISSION CODE

# prediction_total= []
# ids_total = []
# for subject in SUBJECTS:
#     test_features_raw = []
#     train_data_paths = glob(TRAIN_DATA_PATH % (subject))
#     test_data_paths =  glob(TEST_DATA_PATH % (subject))
  
#     x_train, y_train = read_training_data(train_data_paths)
#     x_test, ids = read_test_data(test_data_paths)
#     ids_total.append(ids)
    
    
    
    #Below you can put preprocessing functions. You can make a decision in which method to use
    #by commenting certain lines or not.#     x_train = butter_worth(x_train, CUTT_OFF_FREQUENCY, SAMPLE_FREQUENCY, ORDER)
#     x_test = butter_worth(x_test, CUTT_OFF_FREQUENCY, SAMPLE_FREQUENCY, ORDER)
#     x_train, pca_model = pca_transform(x_train, PCA_COMPONENTS, None)
#     x_test = pca_transform(x_test, PCA_COMPONENTS, pca_model)
#     x_train, scaler = scaler_transform(x_train, None)
#     x_test = scaler_transform(x_test, scaler)
#     #############################################
    
#     predictions = np.empty((x_test.shape[0],6))
#     #Below you define the model you want to use.
#     logreg = LogisticRegression()
#     for i in range(N_LABELS):
#         print('Train subject %d, class %s' % (subject, COLUMNS[i]))
#         model = train_model(x_train, y_train[:,i], logreg, SUBSAMPLE)
#         predictions[:,i] = make_prediction(x_test, model)
        
#     prediction_total.append(predictions)

# make_submission_file(SUBMISSION_NAME, ids_total, prediction_total, COLUMNS, SUBMISSION_FOLDER)